<a href="https://colab.research.google.com/github/yejinkyo/retention-ai/blob/main/clean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. 필요한 라이브러리 설치

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

# 1. 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
telco_df = pd.read_excel('/content/drive/MyDrive/25-2 캡스톤 디자인/Telco_customer_churn.xlsx')

In [ ]:
# 예진 경로

telco_df = pd.read_excel('/content/drive/MyDrive/25-2 캡스톤 디자인/Telco_customer_churn.xlsx')

In [ ]:
telco_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

In [ ]:
telco_df.describe()

,Count,Zip Code,Latitude,Longitude,Tenure Months,Monthly Charges,Churn Value,Churn Score,CLTV
count,7043.0,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,1.0,93521.964646,36.282441,-119.798880,32.371149,64.761692,0.265370,58.699418,4400.295755
std,0.0,1865.794555,2.455723,2.157889,24.559481,30.090047,0.441561,21.525131,1183.057152
min,1.0,90001.000000,32.555828,-124.301372,0.000000,18.250000,0.000000,5.000000,2003.000000
25%,1.0,92102.000000,34.030915,-121.815412,9.000000,35.500000,0.000000,40.000000,3469.000000
50%,1.0,93552.000000,36.391777,-119.730885,29.000000,70.350000,0.000000,61.000000,4527.000000
75%,1.0,95351.000000,38.224869,-118.043237,55.000000,89.850000,1.000000,75.000000,5380.500000
max,1.0,96161.000000,41.962127,-114.192901,72.000000,118.750000,1.000000,100.000000,6500.000000



1.   7,043개의 행과 33개의 열로 구성됨
2.   Total Charge는 object로 되어있음 -> 숫자로 변경해야함
3.   chrun reason에 5174개 결측치 존재
4.   이상치, 결측치 확인


# 2. 새로운 칼럼 생성

##a. CLTV_Monthly

*   현재 매출 대비 장기 가치를 상대적으로 평가
*   CLTV_Monthly = CLTV / Monthly Charges



In [ ]:
telco_df['CLTV_Monthly'] = telco_df['CLTV']/telco_df['Monthly Charges']

In [ ]:
telco_df['CLTV_Monthly'].head()

,CLTV_Monthly
0,60.148561
1,38.203678
2,53.908680
3,47.738550
4,51.494696


##b. Monthly_Services

*   서비스당 평균 요금

*   Monthly_Services = Monthly Charges / Total Services




In [ ]:
service_cols = [
    'Phone Service',
    'Multiple Lines',
    'Internet Service',
    'Online Security',
    'Online Backup',
    'Device Protection',
    'Tech Support',
    'Streaming TV',
    'Streaming Movies'
]

def count_services(row):
    count = 0
    if row['Phone Service'] == "Yes":
        count += 1
    if row['Multiple Lines'] == "Yes":
        count += 1
    if row['Internet Service'] in ["DSL", "Fiber optic"]:
        count += 1
    for col in ['Online Security', 'Online Backup', 'Device Protection',
                'Tech Support', 'Streaming TV', 'Streaming Movies']:
        if row[col] == "Yes":
            count += 1
    return count

telco_df['Total_Services'] = telco_df.apply(count_services, axis=1)

In [ ]:
telco_df.head(3)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason,CLTV_Monthly,Total_Services
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer,60.148561,4
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Electronic check,70.70,151.65,Yes,1,67,2701,Moved,38.203678,2
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Electronic check,99.65,820.5,Yes,1,86,5372,Moved,53.908680,6


In [ ]:
telco_df['Monthly_Services'] = telco_df['Monthly Charges']/telco_df['Total_Services']

In [ ]:
telco_df.head(3)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason,CLTV_Monthly,Total_Services,Monthly_Services
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,53.85,108.15,Yes,1,86,3239,Competitor made better offer,60.148561,4,13.462500
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,70.70,151.65,Yes,1,67,2701,Moved,38.203678,2,35.350000
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,99.65,820.5,Yes,1,86,5372,Moved,53.908680,6,16.608333


##c. Early_Termination_Ratio

요금과 계약 기간 간의 관계 \
Tenure_Ratio (계약 충족 비율) = TenureMonths / ContractLengthNum \
Early_Termination_Ratio (조기 이탈 플래그 ) cutoff를 0.7로 설정해서 \
0 = "No" (조기해지 위험 아님) \
1 = "Yes" (조기해지 위험)


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
#계약기간 숫자 변환
#ContractLengthNum

contract_map = {
    'Month-to-month': 1,
    'One year': 12,
    'Two year': 24
}
telco_df['ContractLengthNum'] = telco_df['Contract'].map(contract_map)

display(telco_df[['Tenure Months','Contract', 'ContractLengthNum']])

,Tenure Months,Contract,ContractLengthNum
0,2,Month-to-month,1
1,2,Month-to-month,1
2,8,Month-to-month,1
3,28,Month-to-month,1
4,49,Month-to-month,1
...,...,...,...
7038,72,Two year,24
7039,24,One year,12
7040,72,One year,12
7041,11,Month-to-month,1


In [ ]:
# 계약기간 대비 실제 유지율
telco_df['Tenure_Ratio'] = telco_df['Tenure Months'] / telco_df['ContractLengthNum'].replace(0, 1)
display(telco_df[['Tenure_Ratio']])

,Tenure_Ratio
0,2.00
1,2.00
2,8.00
3,28.00
4,49.00
...,...
7038,3.00
7039,2.00
7040,6.00
7041,11.00


In [ ]:
# 계약 충족 비율
telco_df['Fulfillment_Ratio'] = telco_df['Tenure Months'] / telco_df['ContractLengthNum'].replace(0, 1)
display(telco_df['Fulfillment_Ratio'])

,Fulfillment_Ratio
0,2.00
1,2.00
2,8.00
3,28.00
4,49.00
...,...
7038,3.00
7039,2.00
7040,6.00
7041,11.00


In [ ]:
# 조기 이탈 플래그 (cutoff를 0.8로 설정: 약정의 80% 미만 채운 고객)
telco_df['Early_Termination_Flag'] = np.where(
    (telco_df['Contract'] != 'Month-to-Month') & (telco_df['Fulfillment_Ratio'] < 0.8),
    1,
    0
)

display(telco_df['Early_Termination_Flag'].value_counts())

,count
Early_Termination_Flag,
0,6854
1,189


##d. High_ChargePerService_Flag

요금과 서비스 번들 \
같은 요금에 서비스 개수에 따라 만족도가 변화 \
Charge_Per_Service (서비스 대비 요금 비율) = TotalServices / Monthly Charges \
High_ChargePerService_Flag (고위험 플래그) \
0 = "No" -> 서비스 대비 요금이 낮거나 평균 수준 (가성비 양호, 이탈 위험 낮음) \
1 ="Yes" -> 서비스 대비 요금이 상위 25% 이상으로 높음 (가성비 불만 가능성, 이탈 위험 ↑)

In [ ]:
#문자열 표준화
def norm_str(s: pd.Series) -> pd.Series:
  return s.astype(str).str.strip().str.lower()

# Yes/No 계열 표준화
for col in [
    'Phone Service','Multiple Lines','Online Security','Online Backup',
    'Device Protection','Tech Support','Streaming TV','Streaming Movies',
    'Paperless Billing'
]:
    telco_df[col] = norm_str(telco_df[col])

In [ ]:
# Internet Service 표준화 (no / dsl / fiber optic / cable)
telco_df['Internet Service'] = norm_str(telco_df['Internet Service'])

#각 서비스 가입 여부를 0/1로 변환
telco_df['PhoneService_bin']      = (telco_df['Phone Service']      == 'yes').astype(int)
telco_df['MultipleLines_bin']     = (telco_df['Multiple Lines']     == 'yes').astype(int)
# 인터넷은 'no'가 아니면 가입(DSL/Fiber optic/Cable)
telco_df['InternetService_bin']   = (telco_df['Internet Service']   != 'no').astype(int)
telco_df['OnlineSecurity_bin']    = (telco_df['Online Security']    == 'yes').astype(int)
telco_df['OnlineBackup_bin']      = (telco_df['Online Backup']      == 'yes').astype(int)
telco_df['DeviceProtection_bin']  = (telco_df['Device Protection']  == 'yes').astype(int)
telco_df['TechSupport_bin']       = (telco_df['Tech Support']       == 'yes').astype(int)
telco_df['StreamingTV_bin']       = (telco_df['Streaming TV']       == 'yes').astype(int)
telco_df['StreamingMovies_bin']   = (telco_df['Streaming Movies']   == 'yes').astype(int)

#총 서비스 개수
telco_df['TotalServices'] = (
    telco_df['PhoneService_bin'] + telco_df['MultipleLines_bin'] +
    telco_df['InternetService_bin'] + telco_df['OnlineSecurity_bin'] +
    telco_df['OnlineBackup_bin'] + telco_df['DeviceProtection_bin'] +
    telco_df['TechSupport_bin'] + telco_df['StreamingTV_bin'] +
    telco_df['StreamingMovies_bin']
)

In [ ]:
telco_df['Charge_Per_Service'] = (
    telco_df['Monthly Charges'] / telco_df['TotalServices'].replace(0, 1)
)

q75 = telco_df['Charge_Per_Service'].quantile(0.75)
telco_df['High_ChargePerService_Flag'] = (telco_df['Charge_Per_Service'] > q75).astype(int)

display(telco_df[['Monthly Charges','TotalServices','Charge_Per_Service','High_ChargePerService_Flag']])

,Monthly Charges,TotalServices,Charge_Per_Service,High_ChargePerService_Flag
0,53.85,4,13.462500,0
1,70.70,2,35.350000,1
2,99.65,6,16.608333,0
3,104.80,7,14.971429,0
4,103.70,7,14.814286,0
...,...,...,...,...
7038,21.15,1,21.150000,1
7039,84.80,8,10.600000,0
7040,103.20,7,14.742857,0
7041,29.60,2,14.800000,0


##e. Actual Monthly Charge: 실제 월평균 비용
- 총 누적 금액에 비해 월 금액이 크면 해당 달에 과금을 하게 된 것
- 총 누적 금액(Total Charges)/통신사 사용 기간(Tenure Months)

In [ ]:
# Total Charges 숫자형으로 변환
telco_df['Total Charges'] = pd.to_numeric(telco_df['Total Charges'], errors='coerce')

# tenure가 0인 고객은 대부분 신규 고객이므로, 월평균 비용을 0으로 처리함.
telco_df['Actual Monthly Charge'] = np.where(
    telco_df['Tenure Months'] == 0,
    0,
    telco_df['Total Charges'] / telco_df['Tenure Months']
)

# 데이터 확인
print(telco_df[['Total Charges', 'Tenure Months', 'Actual Monthly Charge']].head())

   Total Charges  Tenure Months  Actual Monthly Charge
0         108.15              2              54.075000
1         151.65              2              75.825000
2         820.50              8             102.562500
3        3046.05             28             108.787500
4        5036.30             49             102.781633


##f. Loyalty_Ratio: 비용 대비 충성도 비율
- 총 누적 금액(Total Charges)/월별 총 금액(Monthly Charge)

In [ ]:
telco_df['Loyalty_Ratio'] = np.where(
    telco_df['Monthly Charges'] == 0,
    np.nan,
    telco_df['Total Charges'] / telco_df['Monthly Charges']
)

# 데이터 확인
print(telco_df[['Total Charges', 'Monthly Charges', 'Loyalty_Ratio']].head())

   Total Charges  Monthly Charges  Loyalty_Ratio
0         108.15            53.85       2.008357
1         151.65            70.70       2.144979
2         820.50            99.65       8.233818
3        3046.05           104.80      29.065363
4        5036.30           103.70      48.566056


In [ ]:
telco_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 54 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CustomerID                  7043 non-null   object 
 1   Count                       7043 non-null   int64  
 2   Country                     7043 non-null   object 
 3   State                       7043 non-null   object 
 4   City                        7043 non-null   object 
 5   Zip Code                    7043 non-null   int64  
 6   Lat Long                    7043 non-null   object 
 7   Latitude                    7043 non-null   float64
 8   Longitude                   7043 non-null   float64
 9   Gender                      7043 non-null   object 
 10  Senior Citizen              7043 non-null   object 
 11  Partner                     7043 non-null   object 
 12  Dependents                  7043 non-null   object 
 13  Tenure Months               7043 

In [ ]:
telco_df['Loyalty_Ratio'] = telco_df['Loyalty_Ratio'].fillna(0)

In [ ]:
telco_df['Total Charges'] = telco_df['Total Charges'].fillna(0)

# 3. 연속형, 범주형 변수 처리


## a. 범주형 변수 처리

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
X = telco_df.drop('Churn Label', axis=1)
y = telco_df['Churn Label']

In [ ]:
# train, test셋 8:2로 나눔
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 다중 범주형 변수만 따로 분리
cols_to_encode = ['Multiple Lines',
                  'Internet Service',
                  'Online Security',
                  'Online Backup',
                  'Device Protection',
                  'Tech Support', 'Streaming TV', 'Streaming Movies','Contract', 'Payment Method', 'Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Paperless Billing']

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [ ]:
encoder.fit(X_train[cols_to_encode])

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [ ]:
encoded_train = encoder.transform(X_train[cols_to_encode])

In [ ]:
encoded_test = encoder.transform(X_test[cols_to_encode])

In [ ]:
encoded_cols = encoder.get_feature_names_out(cols_to_encode)

In [ ]:
df_encoded_train = pd.DataFrame(encoded_train, columns=encoded_cols, index=X_train.index)
df_encoded_test = pd.DataFrame(encoded_test, columns=encoded_cols, index=X_test.index)

In [ ]:
X_train_telco = X_train.drop(columns=cols_to_encode).join(df_encoded_train)
X_test_telco = X_test.drop(columns=cols_to_encode).join(df_encoded_test)

In [ ]:
X_train_telco.head(2)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Tenure Months,...,Senior Citizen_No,Senior Citizen_Yes,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,Phone Service_no,Phone Service_yes,Paperless Billing_no,Paperless Billing_yes
2142,9571-EDEBV,1,United States,California,San Diego,92119,"32.802959, -117.027095",32.802959,-117.027095,71,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
1623,7562-GSUHK,1,United States,California,Jamestown,95327,"37.84771, -120.486589",37.847710,-120.486589,16,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


##b. 연속형 변수 처리

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
numerical_cols = ['Monthly Charges', 'Total Charges', 'Churn Score', 'CLTV','CLTV_Monthly','Monthly_Services','Fulfillment_Ratio','Charge_Per_Service','Actual Monthly Charge','Loyalty_Ratio']

#StandardScaler
standard_scaler = StandardScaler()
X_train_telco[numerical_cols] = standard_scaler.fit_transform(X_train_telco[numerical_cols])
X_test_telco[numerical_cols] = standard_scaler.transform(X_test_telco[numerical_cols])

#MinMaxScaler
minmax_scaler = MinMaxScaler()
X_train_telco[numerical_cols] = minmax_scaler.fit_transform(X_train_telco[numerical_cols])
X_test_telco[numerical_cols] = minmax_scaler.transform(X_test_telco[numerical_cols])

In [ ]:
#확인 코드 표준화
X_train_standard = X_train_telco.copy()
stats_standard = X_train_standard[numerical_cols].describe()
display_standard = stats_standard.loc[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]

#확인 코드 정규화
X_train_minmax = X_train_telco.copy()
stats_minmax = X_train_minmax[numerical_cols].describe()
display_minmax = stats_minmax.loc[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]

In [ ]:
X_train_telco.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Tenure Months,...,Senior Citizen_No,Senior Citizen_Yes,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,Phone Service_no,Phone Service_yes,Paperless Billing_no,Paperless Billing_yes
2142,9571-EDEBV,1,United States,California,San Diego,92119,"32.802959, -117.027095",32.802959,-117.027095,71,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
1623,7562-GSUHK,1,United States,California,Jamestown,95327,"37.84771, -120.486589",37.847710,-120.486589,16,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
6074,2325-WINES,1,United States,California,Littlerock,93543,"34.505273, -117.955054",34.505273,-117.955054,32,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1362,9381-NDKME,1,United States,California,El Monte,91731,"34.079934, -118.046695",34.079934,-118.046695,24,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
6754,2164-SOQXL,1,United States,California,Los Angeles,90043,"33.988543, -118.334081",33.988543,-118.334081,20,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0


##c. Churn Label 원핫인코딩


*   yes = 1, no=0



In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
y_train = le.fit_transform(y_train)

In [ ]:
y_train = pd.DataFrame(y_train, columns=['Churn Label'])

In [ ]:
y_test = le.transform(y_test)

In [ ]:
y_test = pd.DataFrame(y_test, columns=['Churn Label'])

In [ ]:
y_train.value_counts()

,count
Churn Label,
0,4165
1,1469


# 군집화 위해서 다시 합치기

In [ ]:
combined_X = pd.concat([X_train_telco, X_test_telco])

In [ ]:
#군집화는 비지도 학습이라 y는 필요없을 듯
combined_y = pd.concat([y_train, y_test])

In [ ]:
#인덱스 재정렬
combined_X = combined_X.reset_index(drop=True)
combined_y = combined_y.reset_index(drop=True)

In [ ]:
#y(Chrun Label)도 포함된 데이터
telco_final_df = pd.concat([combined_X, combined_y], axis=1)

# 최종 데이터 저장

In [ ]:
#y(Chrun Label)도 포함된 데이터
telco_final_df.to_csv('telco_clean_data.csv', index=False)

In [4]:
print("코랩 연동 테스트")

코랩 연동 테스트


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/2025_캡스톤디자인/retention-ai

/content/drive/MyDrive/2025_캡스톤디자인/retention-ai


In [5]:
!git config --global user.email 'p2006655@gmail.com'
!git config --global user.name 'yejinkyo'

In [6]:
!git status

Refresh index: 100% (6/6), done.
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   clean_data.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
